In [1]:
import numpy as np

In [2]:
# (ij|kl) in chemists' notation
h2e = np.load("2e.npy")

# scf coefficnets, the 1st index is atomic basis, the 2nd index is the molecular orbital
# `scf.npz` is calculated from SimpleSCF.ipynb
scf_npz = np.load("scf.npz")
scf_c = scf_npz["mo_coef"]
scf_e = scf_npz["mo_energy"]
scf_e_tot = scf_npz["e_tot"]

# basis transformation to canonical HF orbital basis
for i in range(4):
    h2e = np.tensordot(h2e, scf_c, axes=1).transpose(3, 0, 1, 2)
# transform to physics' notation, <ik|jl> = (ij|kl)
h2e = h2e.transpose(0, 2, 1, 3)

In [3]:
# number of orbitals
K = h2e.shape[0]
# number of electrons
N = 6 * 6 + 6

In [4]:
# Eq.6.74

# <ab|rs>
h2e_abrs = h2e[:N//2, :N//2, N//2:, N//2:]
# <rs|ab>
h2e_rsab = h2e[N//2:, N//2:, :N//2, :N//2]
# <rs|ba>
h2e_rsba = h2e[N//2:, N//2:, :N//2, :N//2]

ea = scf_e[:N//2].reshape(-1, 1, 1, 1)
eb = scf_e[:N//2].reshape(1, -1, 1, 1)
er = scf_e[N//2:].reshape(1, 1, -1, 1)
es = scf_e[N//2:].reshape(1, 1, 1, -1)
denominator = ea + eb - er - es

e2 = 2 * (h2e_abrs / denominator).ravel() @ h2e_rsab.transpose(2, 3, 0, 1).ravel() - (h2e_abrs / denominator).ravel() @ h2e_rsba.transpose(3, 2, 0, 1).ravel()

In [5]:
scf_e_tot + e2, e2

(-228.24274700716512, -0.35333708342039505)